In [1]:
from scipy.stats import multivariate_normal as mvn
from scipy.stats import norm
from scipy.stats import chi2
import numpy as np
import pandas as pd
from scipy import optimize
import math

In [2]:
# Copula
# V1, V2 shall be cdf of original prob function
# rho is the copula correlation
V1 = 1-math.exp(-3*1)
V2 = 1-math.exp(-4*0.5)
rho = 0.5



print("V1:", round(V1, 4))
print("V2:", round(V2, 4))


U1 = norm.ppf(V1)
U2 = norm.ppf(V2)

print("U1:", round(U1, 4))
print("U2:", round(U2, 4))
print()

# copula assumes a N(0, 1) for partial u1, u2.
cop_corr = mvn.cdf([U1, U2],
                    mean=[0, 0],  
                    cov=[[1, rho],
                         [rho, 1]])

print("Joint Probability by Copula:", round(cop_corr, 6))

V1: 0.9502
V2: 0.8647
U1: 1.6469
U2: 1.1015

Joint Probability by Copula: 0.838175


In [3]:
# WCDR
# PD: one year prob of default
# sig: estimation worst case (confidence level)
# rho: copula correlation

rho = 0.2
sig = 99.97 / 100
PD = 1.2 / 100


_u = (norm.ppf(PD) + math.sqrt(rho) * norm.ppf(sig)) / math.sqrt(1 - rho)
WCDR = norm.cdf(_u)

print("WCDR:", round(WCDR, 6))

WCDR: 0.20962


In [4]:
# EWMA(1)
# std_0 is DAILY std.

lamb = 0.94

std_0 = 1.3 / 100
price_0 = 300
price_1 = 303


u_0 = price_1 / price_0 - 1
_var_1 = lamb * math.pow(std_0, 2) + (1 - lamb) * math.pow(u_0, 2)
std_1 = math.sqrt(_var_1)

print("std_1:", round(std_1, 6))

std_1: 0.01284


In [5]:
# ARCH(m): var_1 = gamma*V_L + sigma(alpha_i * std_0**2)

# GARCH(1, 1) Classic
# std_0 is DAILY std.

alpha = 0.13
beta = 0.86
w = 0.000002

std_0 = 1.6 / 100
price_0 = 300
price_1 = price_0 * (1 + 0.01)



u_0 = price_1 / price_0 - 1
_var_1 = w + alpha * math.pow(u_0, 2) + beta * math.pow(std_0, 2)
std_1 = math.sqrt(_var_1)

var_L = w / (1 - alpha - beta)

print("std_one:", round(std_1, 6))
print("std_L:", round(math.sqrt(var_L), 6))

std_one: 0.015335
std_L: 0.014142


In [6]:
# GARCH(1, 1) Covariance
# std_0 is DAILY std.

alpha = 0.04
beta = 0.93
w = 0.000003

std_x = 0.08
std_y = 0.02
corr_xy = -0.8

cov_0 = corr_xy * (std_x * std_y)
price_x0 = 100
price_x1 = 102
price_y0 = 100
price_y1 = 102.5

u_x0 = price_x1 / price_x0 - 1
u_y0 = price_y1 / price_y0 - 1
cov_1 = w + alpha * u_x0 * u_y0 + beta * cov_0
var_L = w / (1 - alpha - beta)

print("cov_1:", round(cov_1, 6))
print("var_L:", round(var_L, 6))

cov_1: -0.001167
var_L: 0.0001


In [7]:
# GARCH(1, 1) Long Forecast
# std_0 is DAILY std.

alpha = 0.04
beta = 0.94
w = 0.000002

std_0 = 1.9 / 100
t = 40


var_L = w / (1 - alpha - beta)

_var_t = var_L + math.pow(alpha + beta, t) * (math.pow(std_0, 2) - var_L)
std_t = math.sqrt(_var_t)

print("std_L:", round(math.sqrt(var_L), 6))
print("estimate of", t, "days DAILY std:", round(std_t, 6))

std_L: 0.01
estimate of 40 days DAILY std: 0.014708


In [8]:
# GARCH(1, 1) change in Option's annual volatility
# std_0 is DAILY std.

alpha = 0.04
beta = 0.94
w = 0.000002

std_0 = 0.012
t = 20

a = math.log(1 / (alpha + beta))
var_L = w / (1 - alpha - beta)


_var_t_an = 252 * (var_L + (1 - math.exp(-a * t)) / (a * t) * (math.pow(std_0, 2) - var_L))
std_t_an = math.sqrt(_var_t_an)

print("std_L:", round(math.sqrt(var_L), 6))
print("Annualized std to price", t, "days options:", round(std_t_an, 6))

std_L: 0.01
Annualized std to price 20 days options: 0.185261


In [9]:
# GARCH(1, 1) Two Vol.
# std_0, std_1 are DAILY std.

alpha = 0.03
beta = 0.95
w = 0.000002

std_0 = 1.5 / 100
std_1 = 2 / 100
t = 20


a = math.log(1 / (alpha + beta))
var_L = w / (1 - alpha - beta)
_var_t_an = 252 * (var_L + (1 - math.exp(-a * t)) / (a * t) * (math.pow(std_0, 2) - var_L))
std_tbar_an = math.sqrt(_var_t_an)
_var_t = var_L + math.pow(alpha + beta, t) * (math.pow(std_0, 2) - var_L)
delta_std_t_an = 252 * ((1 - math.exp(-a * t)) / (a * t)) * (std_0 / std_tbar_an) * (std_1 - std_0)


print("t:", t, '\n',
      "change of std:", round(delta_std_t_an, 6), '\n', 
      "new ANNUALIZED std on option:", round(std_tbar_an + delta_std_t_an, 6))


t: 20 
 change of std: 0.068771 
 new ANNUALIZED std on option: 0.294853
